In [1]:
## import numpy as np # linear algebra
from matplotlib import pyplot as plt
#We need these in this file:
import sys
sys.path.append('cropping')
sys.path.append('scripts')
import load_read_name_extractor as lrne
import SVM_classifier_general as svm_general
import DeepFunctions as df

#Taget fra længere nede i koden:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from PIL import Image
import random
import itertools
import numpy as np
from random import shuffle
from IPython.display import display
import os
import json


c:\Users\marti\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Remember to change the cuda:0 value. Should match a free GPU on the cluster or if using local probably just cuda:0.

In [2]:
# Call the function to list GPU names
df.list_gpu_names()
#R
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Number of available GPUs: 2
GPU 0: NVIDIA GeForce RTX 2070 SUPER
GPU 1: NVIDIA GeForce GTX 1050 Ti
cuda:0


In [3]:
trainDataloader = torch.load('data/train_loader.pth')
valDataloader = torch.load('data/val_loader.pth')
testDataloader = torch.load('data/test_loader.pth')

In [4]:
lenthdataset = torch.load('data/lengthsdataset.pth')

In [5]:
print(len(trainDataloader))
print(len(valDataloader))
print(len(testDataloader))

<class 'torch.utils.data.dataloader.DataLoader'>
117


In [6]:

#We have ignored class 9 since there were only 1 image with that class. Therfore we have 9 classes.
num_classes = 9

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3,stride=1):
        super(ResidualBlock, self).__init__()

        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection (identity mapping)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Add the shortcut connection
        out += self.shortcut(residual)
        out = self.relu(out)

        return out

In [8]:
kernel_size = 3
padding = 0
# define network
conv1toconv2 = 16
conv2toconv3 = 32
conv3toconv4 = 64
conv4toconv5 = 128
conv5tores1 = 256
res1tores2 = 256
res2tores3 = 256
res3tores4 = 256
res4tores5 = 256
res5tores6 = 256
res6tolin1 = 256
lin = 256

class NeuralNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5, momentum=0.1):
        super(NeuralNetwork, self).__init__()
        
        self.conv1 = nn.Conv2d(3, conv1toconv2, kernel_size, padding=padding)
        self.bn1 = nn.BatchNorm2d(conv1toconv2, momentum = momentum)
        self.conv2 = nn.Conv2d(conv1toconv2, conv2toconv3, kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm2d(conv2toconv3)
        self.conv3 = nn.Conv2d(conv2toconv3, conv3toconv4, kernel_size, padding=padding)
        self.bn3 = nn.BatchNorm2d(conv3toconv4)
        self.conv4 = nn.Conv2d(conv3toconv4, conv4toconv5, kernel_size, padding=padding)
        self.bn4 = nn.BatchNorm2d(conv4toconv5)
        self.conv5 = nn.Conv2d(conv4toconv5, conv5tores1, kernel_size, padding=padding)
        self.bn5 = nn.BatchNorm2d(conv5tores1)
        
        self.residual1 = ResidualBlock(conv5tores1, res1tores2)
        self.residual2 = ResidualBlock(res1tores2, res2tores3)
        self.residual3 = ResidualBlock(res2tores3, res3tores4)
        self.residual4 = ResidualBlock(res4tores5, res4tores5)
        self.residual5 = ResidualBlock(res4tores5, res5tores6)
        self.residual6 = ResidualBlock(res5tores6, res6tolin1)
        
        self.pool = nn.AvgPool2d(2, 2)
        self.poolmax = nn.MaxPool2d(2,2)
        self.flatten = nn.Flatten()
        self.FC1 = nn.Linear(6 * res6tolin1, lin)
        self.bn7 = nn.BatchNorm1d(lin)
        self.FC2 = nn.Linear(lin,lin)
        self.bn8 = nn.BatchNorm1d(lin)
        self.FC3 = nn.Linear(lin, num_classes)
        self.bn9 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        #print(x.shape)
        x = self.poolmax(x)
        
        #print(x.shape)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        #print(x.shape)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)
        
                                  
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool(x)
        #print(x.shape)
        
        x = self.conv5(x)
        x = self.bn5(x)
        x = F.relu(x)
        x = self.pool(x)
        #print(x.shape)
        
        x = self.residual1(x)
        x = self.residual2(x)
        x = self.residual3(x)
        x = self.residual4(x)
        x = self.residual5(x)
        x = self.residual6(x)
        
        x = self.pool(x)
        
        x = self.flatten(x)
        x = self.FC1(x)
        x = self.bn7(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.FC2(x)
        x = self.bn8(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.FC3(x)
        return x

In [9]:
#%%capture output
done=False
learning_rates = [0.00001]
patience_values = [7]
weight_decay_values =[0]
label_smoothing_values =[0.2]
factor_values=[0.1,0.5]
message = "Standard"

# Set the number of epochs
num_epochs = 120
totalAcc = 0

trainAccList = []
valAccList = []
trainLossList = []
valLossList = []

# Variables for early stopping

best_epoch = 0
best_model_state = None
filename = None
start_scheduler_epoch = 0

# Training loop + tuning
for learning_rate in learning_rates:
    for weight_decay in weight_decay_values:
        for patience in patience_values:
            for label_smoothing in label_smoothing_values:
                for factor in factor_values:
                    net = NeuralNetwork()
                    print("start")
                    net.to(device)
                    #print(net)
                    #learning_rate = 0.001
                    # Define the optimizer
                    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing).to(device)
                    optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
                    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=patience, verbose=True, factor=factor)
                    best_val_acc = 0.0
                    trainAccList = []
                    valAccList = []
                    trainLossList = []
                    valLossList = []
                    # Training loop
                    for epoch in range(num_epochs):
                        running_loss = 0.0
                        totalAcc = 0.0
                        val_running_loss = 0.0
                        val_totalAcc = 0.0
                        true_labels = []
                        predicted_labels = []
                        for i, data in enumerate(trainDataloader, 0):
                            net.train()
                            # Get the inputs and labels
                            inputs, labels = data

                            inputs, labels = inputs.to(device), labels.to(device)

                            # Zero the parameter gradients
                            optimizer.zero_grad()

                            # Forward pass
                            outputs = net(inputs)
                            # Move tensors to CPU before extending lists
                            _, predicted = torch.max(outputs, 1)  # Assuming a classification task with softmax activation
                            true_labels.extend(labels.cpu().numpy())
                            predicted_labels.extend(predicted.cpu().numpy())
                            loss = criterion(outputs, labels)

                            # Backward pass and optimization
                            loss.backward()
                            optimizer.step()

                            # Print statistics
                            running_loss += loss.item()
                            del(inputs)
                            del(labels)
                            # Convert lists to NumPy arrays
                        true_labels = np.array(true_labels)
                        predicted_labels = np.array(predicted_labels)
                        totalAcc = accuracy_score(true_labels, predicted_labels)

                        true_labels = []
                        predicted_labels = []
                        for i, data in enumerate(valDataloader, 0):
                            net.eval()
                            # Get the inputs and labels
                            inputs, labels = data
                            inputs, labels = inputs.to(device), labels.to(device)
                            # Forward pass
                            outputs = net(inputs)
                            _, predicted = torch.max(outputs, 1)  # Assuming a classification task with softmax activation
                            # Move tensors to CPU before extending lists
                            loss = criterion(outputs, labels)

                            # Print statistics
                            val_running_loss += loss.item()
                            true_labels.extend(labels.cpu().numpy())
                            predicted_labels.extend(predicted.cpu().numpy())
                            del(inputs)
                            del(labels)
                            
                            # Convert lists to NumPy arrays
                        true_labels = np.array(true_labels)
                        predicted_labels = np.array(predicted_labels)
                        val_totalAcc = accuracy_score(true_labels, predicted_labels)
                            
                        if epoch >= start_scheduler_epoch:
                            scheduler.step(val_running_loss)
                        

                        trainAccList.append(totalAcc)
                        valAccList.append(val_totalAcc)
                        trainLossList.append(running_loss/len(trainDataloader))
                        valLossList.append(val_running_loss/len(valDataloader))
                        print('[%d] train loss: %.3f train acc: %.3f val loss: %.3f val acc: %.3f '
                              % (epoch + 1,trainLossList[epoch],  trainAccList[epoch], valLossList[epoch], valAccList[epoch]))
                        running_loss = 0.0
                        totalAcc = 0.0
                        val_running_loss = 0.0
                        val_totalAcc = 0.0
                        current_learning_rate = optimizer.param_groups[0]['lr']

                        # Check for improvement in validation accuracy

                        if valAccList[epoch] > best_val_acc and valAccList[epoch] > 0.30:
                            best_val_acc = valAccList[epoch]
                            best_model_state = net.state_dict()
                            best_epoch = epoch

                        # Break if totalAcc is above 0.95

                    if best_model_state is not None:
                        net.load_state_dict(best_model_state)
                        torch.save(best_model_state, 'models/best_model_stateAcc%.3fepoch%.1f.pt' % (best_val_acc, best_epoch))
                        filename = 'models/best_model_stateAcc%.3fepoch%.1f.pt' % (best_val_acc, best_epoch)
                    best_val_accuracy = 0.0
                    best_epoch = 0
                    best_model_state = None


                    print('Finished training')
                    df.save_training_data_to_file(df.transfer_to_cpu([learning_rate,weight_decay,patience,momentum,label_smoothing,factor,filename]),df.transfer_to_cpu(trainLossList)
                                                      ,df.transfer_to_cpu(trainAccList),df.transfer_to_cpu(valLossList)
                                                      ,df.transfer_to_cpu(valAccList))
done=True

start


[1] train loss: 2.152 train acc: 0.214 val loss: 2.935 val acc: 0.208 
[2] train loss: 2.032 train acc: 0.296 val loss: 2.160 val acc: 0.226 
[3] train loss: 1.990 train acc: 0.313 val loss: 2.017 val acc: 0.294 
[4] train loss: 1.942 train acc: 0.343 val loss: 2.028 val acc: 0.269 
[5] train loss: 1.903 train acc: 0.360 val loss: 2.174 val acc: 0.231 
[6] train loss: 1.871 train acc: 0.389 val loss: 2.055 val acc: 0.258 
[7] train loss: 1.842 train acc: 0.406 val loss: 2.876 val acc: 0.223 
[8] train loss: 1.819 train acc: 0.410 val loss: 2.209 val acc: 0.256 
[9] train loss: 1.797 train acc: 0.433 val loss: 2.312 val acc: 0.232 
[10] train loss: 1.780 train acc: 0.435 val loss: 2.119 val acc: 0.266 
Epoch 00011: reducing learning rate of group 0 to 1.0000e-06.
[11] train loss: 1.754 train acc: 0.452 val loss: 2.499 val acc: 0.202 
[12] train loss: 1.723 train acc: 0.475 val loss: 1.748 val acc: 0.432 
[13] train loss: 1.718 train acc: 0.480 val loss: 1.781 val acc: 0.402 
[14] train 

In [ ]:
#if done:
#	output.show() # displays captured output
#else:
#	print("cell above still seem to be running, wait some more..")

NameError: name 'output' is not defined